In [2]:
import pandas as pd

In [3]:
trend_df = pd.read_parquet("/zfsauton2/home/ekaczmar/representations-in-tsfms-main/representations-in-tsfms-main/hough_transform/datasets/trend.parquet")
sine_df = pd.read_parquet("/zfsauton2/home/ekaczmar/representations-in-tsfms-main/representations-in-tsfms-main/hough_transform/datasets/sine.parquet")




In [5]:
import pandas as pd
import numpy as np

def all_rows_identical(df: pd.DataFrame, rtol=1e-5, atol=1e-8):
    try:
        # Convert to float array safely
        data = df.astype(float).values
        
        # Compare each row to the first one
        return np.allclose(data, data[0], rtol=rtol, atol=atol)
    except Exception as e:
        print("Error during comparison:", e)
        return False

if all_rows_identical(trend_df):
    print("✅ All time series in trend_df are identical.")
else:
    print("❌ Mismatch found in trend_df.")

if all_rows_identical(sine_df):
    print("✅ All time series in sine_df are identical.")
else:
    print("❌ Mismatch found in sine_df.")

Error during comparison: setting an array element with a sequence.
❌ Mismatch found in trend_df.
Error during comparison: setting an array element with a sequence.
❌ Mismatch found in sine_df.


In [9]:
def find_differing_rows(df: pd.DataFrame, rtol=1e-5, atol=1e-8):
    data = df.astype(float).values
    diffs = []
    for i in range(len(data)):
        if not np.allclose(data[i], data[0], rtol=rtol, atol=atol):
            diffs.append(i)
    return diffs

diff_trend = find_differing_rows(trend_df)
diff_sine = find_differing_rows(sine_df)

print(f"Differing rows in trend_df: {diff_trend}")
print(f"Differing rows in sine_df: {diff_sine}")

ValueError: setting an array element with a sequence.

In [10]:
print(type(trend_df.iloc[0, 0]))

<class 'numpy.ndarray'>


In [11]:
data = np.vstack(trend_df.iloc[:, 0].values) 

In [12]:
def find_differing_rows_array(data: np.ndarray, rtol=1e-5, atol=1e-8):
    diffs = []
    for i in range(len(data)):
        if not np.allclose(data[i], data[0], rtol=rtol, atol=atol):
            diffs.append(i)
    return diffs

diff_rows = find_differing_rows_array(data)
print("Differing row indices:", diff_rows)

Differing row indices: []


In [ ]:
def ab_to_r_theta(a_vals, b_vals):
    """
    Convert lines in slope-intercept form (y = a*x + b)
    to Hough space parameters (r, theta).

    Parameters:
        a_vals: array-like of slopes (a)
        b_vals: array-like of intercepts (b)

    Returns:
        r: ndarray of distances from origin
        theta: ndarray of angles (in radians) of normal vector
    """
    a_vals = np.asarray(a_vals, dtype=float)
    b_vals = np.asarray(b_vals, dtype=float)

    # Handle vertical lines (infinite slope)
    vertical_mask = np.isinf(a_vals)
    theta = np.empty_like(a_vals)
    r = np.empty_like(b_vals)

    # General case: a is finite
    finite_mask = ~vertical_mask
    theta[finite_mask] = np.arctan(-1 / a_vals[finite_mask])
    r[finite_mask] = b_vals[finite_mask] * np.sin(theta[finite_mask])

    # Special case: vertical lines → θ = 0, r = x-intercept
    theta[vertical_mask] = 0.0
    r[vertical_mask] = b_vals[vertical_mask]  # interpret b as x-intercept here

    return r, theta

def vector_point_to_ab(v, p):
    """
    Given a 2D direction vector v = (vx, vy)
    and a point p = (px, py), compute the line y = ax + b

    Returns:
        a: slope (or np.inf for vertical line)
        b: intercept (ignored if vertical)
    """
    vx, vy = v
    px, py = p

    if vx == 0:
        a = np.inf
        b = px  # x-intercept
    else:
        a = vy / vx
        b = py - a * px

    return a, b


In [13]:
print(data.shape) 

(512, 512)


In [ ]:
def cartesian_to_hyperspherical2(vector): #for 1 sample for now 
    #vector = vector.detach().cpu().numpy()
    r = np.linalg.norm(vector)
    dim = len(vector)
    angles = np.zeros(dim - 1)
    for i in range(dim - 1):
        if vector[i:].any() != 0: 
            angles[i] = np.arctan2(np.linalg.norm(vector[(i+1):]), vector[i])
        else: 
            angles[i] == 0 
    return r, angles

In [46]:
np.arctan2(1,0)

1.5707963267948966

In [48]:
vector = np.array([0,1])
r, angles = cartesian_to_hyperspherical2(vector)
print(r)
print(angles)

c
1.0
0
1.0
[1.57079633]


In [16]:
np.pi/4

0.7853981633974483